**This document will serve as a step by step report, on how the data was prepared for analysis, and later how it was visualized**

In [8]:
# Relevantni importi
% matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from Orange.data.filter import SameValue
from Orange.data import Table
import numpy as np
from csv import DictReader

In [9]:
# Odprem podatke za branje
crash1_reader = DictReader(open('data/Crash_Data_1.csv', 'rt', encoding='utf-8'))
crash2_reader = DictReader(open('data/Crash_Data_2.csv', 'rt', encoding='utf-8'))
liquor_reader = DictReader(open('data/Liquor_Data.csv', 'rt', encoding='utf-8'))

# crash1_o = Table("data/Crash_Data_1.csv")
# crash2_o = Table("data/Crash_Data_2.csv")
# liquor_o = Table("data/Liquor_Data.csv")

In [10]:
# Pogledam imena polj v podatkih
# print(crash1_o.domain)
# print(crash2_o.domain)
# print(liquor_o.domain)
print(crash1_reader.fieldnames)
print("--------------")
print(crash2_reader.fieldnames)
print("--------------")
print(liquor_reader.fieldnames)

['\ufeffX', 'Y', 'OBJECTID', 'CRASH_KEY', 'CASENUMBER', 'LECASENUM', 'CRASH_DATE', 'CRASH_MONTH', 'CRASH_DAY', 'TIMESTR', 'DISTRICT', 'COUNTY_NUMBER', 'CITYNAME', 'SYSTEMSTR', 'LITERAL', 'FRSTHARM', 'LOCFSTHRM', 'CRCOMNNR', 'MAJCSE', 'DRUGALC', 'ECNTCRC', 'LIGHT', 'CSRFCND', 'WEATHER', 'RCNTCRC', 'RDTYP', 'PAVED', 'WZRELATED', 'CSEV', 'FATALITIES', 'INJURIES', 'MAJINJURY', 'MININJURY', 'POSSINJURY', 'UNKINJURY', 'PROPDMG', 'VEHICLES', 'TOCCUPANTS', 'REPORT', 'XCOORD', 'YCOORD']
--------------
['\ufeffX', 'Y', 'OBJECTID', 'VEH_CRASH_KEY', 'VEH_UNITKEY', 'CASENUMBER', 'DRIVERAGE', 'DRIVERGEN', 'DL_STATE', 'CHARGED', 'ALCRESULT', 'DRUGTEST', 'DRUGRESULT', 'DRIVERCOND', 'VISIONOBS', 'DCONTCIRC1', 'DCONTCIRC2', 'VCONFIG', 'CARGOBODY', 'VYEAR', 'MAKE', 'MODEL', 'STYLE', 'VLP_STATE', 'OCCUPANTS', 'VACTION', 'SEQEVENTS1', 'SEQEVENTS2', 'SEQEVENTS3', 'SEQEVENTS4', 'MOSTHARM', 'SPEEDLIMIT', 'TRAFCONT', 'FIXOBJSTR', 'MOSTDAMAGE', 'DAMAGE', 'CSEVERITY', 'MAJORCAUSE', 'CSURFCOND', 'DRUGALCREL', 'RO

In [11]:
# Pregledam kako vnosi podatkov zgledajo
for crash in crash1_reader:
    print(crash)
    break
print("------------------")
for crash in crash2_reader:
    print(crash)
    break
print("------------------")
for liquor in liquor_reader:
    print(liquor)
    break

OrderedDict([('\ufeffX', '-93.511904910700665'), ('Y', '41.648063161780378'), ('OBJECTID', '1001'), ('CRASH_KEY', '2016902721'), ('CASENUMBER', '2016902721'), ('LECASENUM', '16-452'), ('CRASH_DATE', '2016-01-20T00:00:00.000Z'), ('CRASH_MONTH', '1'), ('CRASH_DAY', '4'), ('TIMESTR', '20:02'), ('DISTRICT', '1'), ('COUNTY_NUMBER', '77'), ('CITYNAME', '132'), ('SYSTEMSTR', 'US 6'), ('LITERAL', 'US 6/HUBBELL AVE/NE RAMP/SPEC CASE/FREDERICK M HUBBELL AVE'), ('FRSTHARM', '33'), ('LOCFSTHRM', '1'), ('CRCOMNNR', '4'), ('MAJCSE', '9'), ('DRUGALC', '8'), ('ECNTCRC', '1'), ('LIGHT', '5'), ('CSRFCND', '1'), ('WEATHER', '1'), ('RCNTCRC', '1'), ('RDTYP', '22'), ('PAVED', '1'), ('WZRELATED', ''), ('CSEV', '5'), ('FATALITIES', '0'), ('INJURIES', '0'), ('MAJINJURY', '0'), ('MININJURY', '0'), ('POSSINJURY', '0'), ('UNKINJURY', '0'), ('PROPDMG', '6000'), ('VEHICLES', '2'), ('TOCCUPANTS', '2'), ('REPORT', '7'), ('XCOORD', '457373'), ('YCOORD', '4610828')])
------------------
OrderedDict([('\ufeffX', '-92.80

**Ker so nefiltrirani podatki različnih velikosti, in imajo različno število vnosov, jih sedaj moram filtrirati po neki kriterijski funkciji**
Crash_Data_1.csv je velik 73 Mb
Crash_Data_2.csv je velik 123 Mb
Liquor_Data.csv je velik 3,47 Gb

In [12]:
# Prvo pregledamo število vnosov v posameznih datotekah,
# Vemo da imajo vsi podatke o datumu, tako da po intuiciji bi lahko
# filtrirali po tem.
crash1_row_count = sum(1 for crash in crash1_reader)
crash2_row_count = sum(1 for crash in crash2_reader)
liquor_row_count = sum(1 for liquor in liquor_reader)

print("Crash1 ima "+ str(crash1_row_count) +" vrstic.") # 73Mb z 350 066 vrstic
print("Crash2 ima "+ str(crash2_row_count) +" vrstic.") # 123Mb z 602 387 vrstic
print("Liquor ima "+ str(liquor_row_count) +" vrstic.") # 3,47 Gb z 12 591 076 vrstic
# Ker je čas branja pre velik, so vrstice zapisane v komentarjih zgoraj

Crash1 ima 350066 vrstic.
Crash2 ima 602387 vrstic.
Liquor ima 12591076 vrstic.
